In [0]:
# Setting spark session
spark.conf.set(
    "fs.azure.account.key.datalakedemomy.blob.core.windows.net",
    "28DDU8+w0matKyqgOdDdDkhAf7JEfdeyrAjNnN2onOx322kbS7pvKtha875nVR9juWQPhy4pZz4V+ASt4/mkbw=="
)

In [0]:
# Step 2: List the files in the Blob Storage using access key
display(dbutils.fs.ls("wasbs://greentaxi@datalakedemomy.blob.core.windows.net/"))



path,name,size,modificationTime
wasbs://greentaxi@datalakedemomy.blob.core.windows.net/2021_Green_Taxi_Trip_Data.csv,2021_Green_Taxi_Trip_Data.csv,102329648,1721891636000


In [0]:
# Step 3: Read a CSV file from Blob Storage using access key
df_csv = spark.read.csv("wasbs://greentaxi@datalakedemomy.blob.core.windows.net/2021_Green_Taxi_Trip_Data.csv", header=True, inferSchema=True)

In [0]:
# green_taxiDF1 = df_csv.repartition(10).write.mode("append").option("header","true").parquet("/Volumes/azuredatabme/default/greentaxi_new")

In [0]:
from pyspark.sql.functions import *


# Step 1 : Create a ReadStream
taxiReadStream = spark.readStream.format("csv").schema(df_csv.schema).option("header","true").load("wasbs://greentaxi@datalakedemomy.blob.core.windows.net/2021_Green_Taxi_Trip_Data.csv")



In [0]:
# step 2 : Convert String Stype to timestamp
taxiReadStream = taxiReadStream.withColumn("lpep_pickup_datetime",to_timestamp(col("lpep_pickup_datetime"),"yyyy-MM-dd HH:mm:ss"))

In [0]:
# step 3 : Add a watermark and perform the aggregation (window of aggregate operation) 
# Use windowing based on event time column
taxitransformation = taxiReadStream\
    .withWatermark("lpep_pickup_datetime", "10 minutes")\
        .groupBy(window(col("lpep_pickup_datetime"), "10 minutes"))\
        .sum("fare_amount")

In [0]:






# step 4 : Write the aggregated data to parquet file
taxiWriteStream = taxitransformation.writeStream.outputMode("append")\
.format("parquet")\
.option("path","wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<path-to-your-folder>") \
.option("checkpointLocation", "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<path-to-your-checkpoint-folder>")\
.start()

taxiWriteStream.awaitTermination()